In [4]:
import pandas as pd

game = pd.read_csv("dataNBA.csv")
game = game.iloc[:, 2:]

game["GAME_DATE"] = pd.to_datetime(game["GAME_DATE"])
game['venue_id'] = game['MATCHUP'].str.contains('vs.').astype(int)
game[['team', 'opponent']] = game['MATCHUP'].str.split(r' @ | vs. ', expand=True)
game["opp_id"] = game["opponent"].astype("category").cat.codes
game["day_code"] = game["GAME_DATE"].dt.day_of_week
game["target"] = (game["WL"] == "W").astype("int")

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

train = game[game["GAME_DATE"] < "2025-02-14"]
test = game[game["GAME_DATE"] >= "2025-02-14"]
predictors = ["venue_id", "opp_id", "day_code"]
rf.fit(train[predictors], train["target"])
preds = rf.predict(test[predictors])

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

team_game = game.groupby("team")

def moyenne(group, c, nc):
  group = group.sort_values("GAME_DATE")
  moyennest = group[c].rolling(5, closed='left').mean()
  moyennest.columns = nc
  group[nc] = moyennest
  group = group.dropna(subset=nc)
  return group

c = ["FGM","FGA","FG_PCT","FG3M","FG3A","FG3_PCT","FTM","FTA","FT_PCT","OREB","DREB","REB","AST","STL","BLK","TOV","PF","PTS"]
nc = [f"{i}_rolling" for i in c]

game_rolling = game.groupby("team").apply(lambda x: moyenne(x, c, nc))
game_rolling = game_rolling.droplevel('team')
game_rolling.index = range(game_rolling.shape[0])

def pred(data, predictors):
  train = data[data["GAME_DATE"] < "2025-02-14"]
  test = data[data["GAME_DATE"] >= "2025-02-14"]
  rf.fit(train[predictors], train["target"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
  precision = precision_score(test["target"], preds)
  return combined, precision

combined, precision = pred(game_rolling, predictors + nc)

combined = combined.merge(game_rolling[["GAME_DATE", "team", "opponent", "PTS"]], left_index=True, right_index=True)

merged = combined.merge(combined.merge(combined, left_on=["team", "GAME_DATE"], right_on=["opponent", "GAME_DATE"]))

merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

merged



<ipython-input-4-e13e8872feab>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  game["GAME_DATE"] = pd.to_datetime(game["GAME_DATE"])
<ipython-input-4-e13e8872feab>:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  game_rolling = game.groupby("team").apply(lambda x: moyenne(x, c, nc))


,actual,predicted,GAME_DATE,team,opponent,PTS,actual_x,predicted_x,team_x,opponent_x,PTS_x,actual_y,predicted_y,team_y,opponent_y,PTS_y
0,0,1,2025-02-20,ATL,ORL,108,0,1,ATL,ORL,108,1,1,ORL,ATL,114
1,0,1,2025-02-20,ATL,ORL,108,0,0,BKN,CLE,97,1,1,CLE,BKN,110
2,0,1,2025-02-20,ATL,ORL,108,1,1,BOS,PHI,124,0,0,PHI,BOS,104
3,0,1,2025-02-20,ATL,ORL,108,0,0,CHA,DEN,115,1,1,DEN,CHA,129
4,0,1,2025-02-20,ATL,ORL,108,0,0,CHI,NYK,111,1,1,NYK,CHI,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,1,0,2025-03-05,WAS,UTA,125,1,0,OKC,MEM,120,0,1,MEM,OKC,103
3812,1,0,2025-03-05,WAS,UTA,125,0,1,POR,BOS,118,1,1,BOS,POR,128
3813,1,0,2025-03-05,WAS,UTA,125,0,1,SAC,DEN,110,1,1,DEN,SAC,116
3814,1,0,2025-03-05,WAS,UTA,125,0,1,UTA,WAS,122,1,0,WAS,UTA,125
